---
# Cleaning Data with Pandas
---
Cleaning data is a huge topic. Some people claim that data scientists spend 80% of their time just cleaning their data. The topics that we will cover here are 

* handling missing values
* removing duplicates
* structuring data
* removing outliers
* finding the right data types

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
%matplotlib inline

# Missing values

A sentinel value is used to represent missing values for numbers. A special combinations of bits represents "Not a number" (NaN). This can be thought of as the numeric equivalent of `None`. In Python `NaN` is available through the `NumPy` and `Pandas` packages. Since Pandas version 1.0, missing values are represented using a special object: `pd.NA`.

This might seems strange at first, but starts to make sense when we think about the semantics of `NaN` or more generally `NA` as a placeholder for a value that is __N__ot **A**vailable. As `NA` just represents any value that we do not know, it would be wrong to say that a value that we do not know equals another value that we do not know. Therefore `NA` can not really equal anything.

To explicitly test for `NA` we need a separate function, that is provided by `pandas`.

In [ ]:
pd.isna(np.nan)

In [ ]:
pd.isna(pd.NA)

In [ ]:
pd.isna(42)

In [ ]:
pd.NA == pd.NA


In [ ]:
np.nan == np.nan

In [ ]:
pd.NA == 42, np.nan == 42

## Handling missing values

In [ ]:
ebola = pd.read_csv('data/ebola_country_timeseries.csv')
ebola.head()

In [ ]:
ebola['Cases_Guinea'].value_counts(dropna=False).head()

In [ ]:
ebola.count()

## Drop
The simplest way to deal with missing data is to simply drop it. However, this can lead to immense data loss, depending on how the data is organized.

In [ ]:
ebola.dropna()

In [ ]:
ebola.dropna(how='all')

## Fill
Instead missing values can be filled in so the rest of the data remains useable. Keep in mind that this always introduces artifacts.

We can fill with a constant value.

In [ ]:
ebola.fillna(0).head()

Or use some more advanced strategies to impute the data, such as compute a per column mean. This can be replaced by every simple summary statistic.

In [ ]:
ebola.mean()

In [ ]:
ebola.fillna(ebola.mean()).head()

Some more advanced techniques, such as the Expectation Maximization (EM) algorithm exist, but are not implemented in `pandas` directly. 

When dealing with serial data, it can make sense to fill missing values with previous or subsequent values. <br>
``method=='ffill'`` will fill values with the previous value in the row, while ``method=='bfill'`` does the reverse.

In [ ]:
ebola.fillna(method='ffill').head()

In [ ]:
ebola.fillna(method='ffill').tail()

In [ ]:
ebola.fillna(method='bfill')

### Advanced filling
Pandas also offers more advanced methods of filling missing values. The [interpolate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas-dataframe-interpolate) function offers different ways of interpolating the missing values.
Some options require scipy as pandas passes them to the according scipy function.
 

In [ ]:
ebola['Cases_Guinea'].head()

In [ ]:
# nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’ will require scipy
# can be installed with pip(conda) install scipy
ebola['Cases_Guinea'].interpolate(method='quadratic').head()

## Calculations with missing values
By default, `NumPy` is very strict about calculations involving `NA` values. Every operation involving `NA`s will result in `NA`. This correct in the sense that the final value of an operation such as `sum` can not be known if even a single value is unknown.

In [ ]:
np.nansum([1, 2, np.nan, 3])

However, from a practical point of view this is not very useful. Therefore pandas takes the approach of gracefully ignoring `NA`s.

In [ ]:
ebola['Cases_Guinea'].sum()

This behavior can be changed if desired.

In [ ]:
ebola['Cases_Guinea'].sum(skipna=False)

# Removing duplicates
Duplicates can arise as part of messy data. It is important to identify them correctly and get rid of them, so they do not influence our statistics.

In [ ]:
df1 = pd.DataFrame({
    'a': [1, 1, 1, 2, 2, 2],
    'b': [10, 20, 30, 40, 50, 50],
})

df1

Check whether a row is a duplicate.

In [ ]:
df1.duplicated()

Drop the duplicate rows.

In [ ]:
df1.drop_duplicates()

Restrict duplicate search to a subset of the columns.

In [ ]:
df1.duplicated(subset='a')

In [ ]:
df1.drop_duplicates(subset='a')

# Data Types

### Finding the right data types
Data can be expressed in several levels of measurement. You need to make sure to find the level of measurement that both semantically and computationally makes sense.

A quick detour to scales of measurement
1. **Nominal level** <br/>
   Numbers only represent categories and nothing more. <br/>
   E.g.: genders, colors<br/>
   You can compute: absolute and relative frequencies, mode   
   
1. **Ordinal level** <br/>
   The order has a meaning.<br/>
   E.g.: school-grades, music charts, answers on a likert scale <br/>
   You can additionally compute: cumulative frequencies, median, quantiles   
   
1. **Interval level** <br/>
   Same intervals should convey same meaning.<br/>
   Ratios of differences can be expressed. <br\>
   E.g.: temperature in celsius, (intelligence) tests<br/>
   You can additionally compute: mean, standard deviation   

1. **Ratio level**<br/>
   Ratios convey meaning and there is a specific 0 point.<br/>
   E.g.: mass, size, time, speed, temperature in Kelvin<br/>
   You can compute: coefficient of variation $c = \frac{s}{\bar X}$, i.e. a normalized standard variation 


## Categorical data
https://pandas.pydata.org/pandas-docs/stable/categorical.html

Using a categorical dtype has several advantages

* it keeps memory usage low
* it makes data useable for numeric modeling algorithms
* it signals to libraries that build on pandas how to treat the data
* it makes the intent clear, that only certain values are allowed in a column and how they relate to each other

The following `Series` could be perfectly represented using categories instead of strings.

In [ ]:
s = pd.Series(['a','b', 'b', 'a', 'c', 'c'])
s

In [ ]:
print(f'The string series is {s.nbytes} bytes big.')

By specifying the `dtype` as "category" the data is automatically converted to a categorical scale.

In [ ]:
s = pd.Series(['a','b', 'b', 'a', 'c', 'c'], dtype='category')
s

In fact the `Series` gets much smaller already. The effect will be stronger on larger `Series`.

In [ ]:
print(f'The categorical series is {s.nbytes} bytes big.')

Categorical data is stored using numeric codes under the hood that map to categories.

In [ ]:
s.cat.categories

In [ ]:
s.cat.codes

Using `dtype='category'` will create unordered categories by default.

In [ ]:
s.cat.ordered

The `.cat` accessor allows changing, renaming and ordering categories.

In [ ]:
s.cat.categories

In [ ]:
s.cat.rename_categories(['x', 'y', 'z'])

A categorical series can also be created from `pd.Categorical`. This allows you set the categories and the ordering explicitly.

In [ ]:
pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c'],
                         ordered=False)

The `Categorical` object can then be passed to the `Series` constructor to obtain a real `Series`.

In [ ]:
cat_series = pd.Series(
    pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c', 'a'],
                         ordered=False)
)
cat_series

### Ordered Categories
What does it mean to have ordered categories?

In [ ]:
cat_series2 = pd.Series(
    pd.Categorical(['c', 'a', 'c', 'b'], categories=['b', 'c', 'a'],
                         ordered=False)
)
cat_series2

In [ ]:
cat_series == cat_series2

In [ ]:
cat_series > cat_series2

In [ ]:
cat_series

In [ ]:
cat_series.mode()

In [ ]:
cat_series.max()

These semantics are lost when you pull out the atomic values. Only the `Series` is categorical, not the single entries.

In [ ]:
cat_series.iloc[0], type(cat_series.iloc[0])

In [ ]:
cat_series.iloc[0] < cat_series.iloc[1]

Now the same for an **ordered** cateogorical `Series`.

In [ ]:
cat_ordered_series = pd.Series(
    pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c', 'a', 'd'],
                         ordered=True)
)
cat_ordered_series

In [ ]:
cat_ordered_series2 = pd.Series(
    pd.Categorical(['c', 'a', 'c', 'b'], categories=['b', 'c', 'a', 'd'],
                    ordered=True)
)
cat_ordered_series2

In [ ]:
cat_ordered_series > cat_ordered_series2

In [ ]:
cat_ordered_series.max()

In [ ]:
cat_ordered_series == cat_ordered_series2

The median does not work on the categorical Series but can be computed using the codes.

In [ ]:
cat_ordered_series

In [ ]:
cat_ordered_series.median()

In [ ]:
cat_ordered_series.cat.codes.median()

If you need to cast existing data, to a categorical type and want to specify the categories and the ordering you can use `pd.CategoricalDtype` to create your own categorical datatype. It works the same way as `pd.Categorical` except that you do not pass the data. The newly created datatype can then be used in an `.astype` cast.

In [ ]:
series = pd.Series(['a', 'b', 'c', 'a'])
series

In [ ]:
from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=['b', 'c', 'a'],
                             ordered=True)
cat_type

In [ ]:
series.astype(cat_type)

Now let's look at a real world dataset and some discretization techniques. The titanic dataset contains features about passengers of the tragic Titanic journey. A common introductory machine learning excercise is to predict survial of the passengers based on the features (see https://www.kaggle.com/c/titanic/data ).

In [ ]:
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

In [ ]:
titanic.dtypes

We include all columns in the description as `object` columns are described differently than `numeric` columns and are excluded from the description by default.

In [ ]:
titanic.describe(include='all')

Let's extend the port of embarkation with the full name to make things a bit more readable. We use a simple merge operation to achieve this (more on this later).

In [ ]:
embarked_map = pd.DataFrame({'Embarked': ['C', 'Q', 'S'],
                             'EmbarkedLong': ['Cherbourg', 'Queenstown', 'Southampton']})
embarked_map

In [ ]:
titanic = titanic.merge(embarked_map).sort_values(by='PassengerId')
titanic.head()

In [ ]:
titanic.dtypes

Since the `EmbarkedLong` column has only three distinct values it is naural to represent it using categories.

In [ ]:
titanic['EmbarkedLong'].unique()

In [ ]:
titanic['EmbarkedLong'] = titanic['EmbarkedLong'].astype('category')
titanic['EmbarkedLong'].head()

In [ ]:
titanic.dtypes

The description for a categorical column is the same as for an `object` column.

In [ ]:
titanic['EmbarkedLong'].describe()

## Discretizing continuous values (Tiling)
Sometimes it makes sense to convert numeric into categorical data. For example, for some problems the exact age of a person might not matter, but only whether the person is underaged or not. This process of conversion is called tiling.

https://pandas.pydata.org/pandas-docs/stable/basics.html#discretization-and-quantiling

In [ ]:
titanic['Age'].describe()

Using `cut` we can discretize numeric values.

In [ ]:
titanic['Age'].head(7)

In [ ]:
pd.cut(titanic['Age'], bins=3).head(7)

By default `cut` will split the data into equally sized intervals. As this seldomly makes sense, we can set the bin edges ourself.

In [ ]:
pd.cut(titanic['Age'], bins=[0, 17, 67, 80], include_lowest=True).head(7)

In [ ]:
pd.cut(titanic['Age'], bins=[0, 17, 67, 80]).value_counts()

If you set the bin edges manually, be sure to cover the whole range as values not falling into an interval will be set to NA.

In [ ]:
pd.cut(titanic['Age'], 
       bins=[64, 66, 67, 80],
       labels=['child', 'grown-up', 'senior']).head(7)

In [ ]:
titanic['Age_coarse'] = pd.cut(titanic['Age'], bins=[0, 17, 67, 80], labels=['child', 'grown-up', 'senior'])
titanic['Age_coarse']

A related function is `qcut`, which cuts at quantiles, meaning it will try to create n evenly sized bins

In [ ]:
pd.qcut(titanic['Age'], 4).head()

## Converting to numeric data
Sometimes numeric data is srewed up somehow. `pd.to_numeric` handles these cases, automatically casting everything to the appropriate type.

In [ ]:
numeric_data = pd.read_csv('data/numeric_data.csv')
numeric_data

In [ ]:
numeric_data.dtypes

In [ ]:
numeric_data['C'].sum()

In [ ]:
numeric_data['B'].astype('int')

In [ ]:
numeric_data['A'].astype('float')

In [ ]:
# the default is errors='raise', which raise an exception
# errors='ignore' will return the input
pd.to_numeric(numeric_data['A'], errors='ignore')

In [ ]:
# error = 'coerce' will coerce into nan
pd.to_numeric(numeric_data['A'], errors='coerce')

In [ ]:
pd.to_numeric(numeric_data['B'], errors='coerce')

In [ ]:
pd.to_numeric(numeric_data['C'], errors='coerce')

to_numeric only works on series, but luckily we can `apply`!

In [ ]:
numeric_data

In [ ]:
numeric_data.apply(pd.to_numeric, errors='coerce').dtypes #keyword-arguments are passed to the respective function

In [ ]:
isinstance(np.nan, float)

## Insertion: Plotting with Pandas

Pandas offers a few Plotting-functions, which base on matplotlib's corresponding functions and internally call these functions themselves. To alter their behaviour, you can pass it an axis to a matplotlib-object: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html

In [ ]:
# density: all bars add up to one
titanic['Age'].hist(density=True)

In [ ]:
# the same in standard matplotlib
plt.hist(titanic['Age'].dropna().values, density=True)
plt.grid()

In [ ]:
# use an already existing axis
fig, ax = plt.subplots()
titanic['Age'].hist(density=True, ax=ax)
ax.set_xlabel('Age');

In [ ]:
titanic['Age_coarse'].value_counts()

The [plot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html) function works on dataframes and series and provides a range of different plots.
By default matplotlib is used as a plotting backend.

In [ ]:
titanic['Age_coarse'].value_counts().plot(kind='pie')

In [ ]:
coarse_age_series = titanic['Age_coarse'].cat.add_categories(['unknown'])
coarse_age_series

In [ ]:
coarse_age_series.cat.categories

In [ ]:
coarse_age_series = coarse_age_series.fillna('unknown')
coarse_age_series

In [ ]:
coarse_age_series.value_counts()

In [ ]:
coarse_age_series.value_counts().plot(kind='pie')

In [ ]:
coarse_age_series.value_counts().plot(kind='barh')

The `plot` function also works on DataFrames.

In [ ]:
titanic[['Age', 'Fare']].plot(kind='scatter', x='Age', y='Fare')

# Merging DataFrames
Merging is a concept that is often used in relational databases. It allows to combine multiple tables into one by joining the columns with respect to the values in a special key column. There are different options this can be achieved.
The `DataFrame.merge` function offers these functionalities borrowed from SQL (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas-dataframe-merge).
![](pictures/merging.png)
 

In [ ]:
df1 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [0, np.pi, 2 * np.pi, 3 * np.pi],
                    'C': ['mouse', 'cat', 'dog', 'fish']})
df1

In [ ]:
df2 = pd.DataFrame({'C': ['mouse', 'horse', 'lizard', 'fish'],
                    'D': [1.0, 1.7, 3.0, 2.1],
                    'E': [1, np.e, np.e ** 2, np.e ** 3]})
df2

## Inner Join
The inner join will take the intersection of the keys.

In [ ]:
df1.merge(df2, how='inner')

## Left Outer Join
The left outer join will keep all values from the left table (the one on which merge is called) and use `NaN` where the right table is missing the respective rows.

In [ ]:
df1.merge(df2, how='left')

## Right Outer Join
The right outer join works just like the left outer join but instead of using all keys from the left table, it uses all keys from the right one.

In [ ]:
df1.merge(df2, how='right')

## Outer Join
The outer join uses all keys that are present in both the left and the right table. Missing rows in either of the tables are filled with `NaN`.

In [ ]:
df1.merge(df2, how='outer')

## Overlapping column names

In [ ]:
df2 = df2.rename(columns={'D': 'A'})
df2

In [ ]:
df1

If it is not implicitly clear on which column the join should occur, we have to tell Pandas which column to use. It can also perform the join on multiple columns but therefore the dtypes of the matched columns inside the two DataFrames must be equal.

In [ ]:
df1.merge(df2, how='inner')

We can explicitly tell Pandas on which column the join should occur using the keyword argument `on`. The `suffixes` parameter allows us to control how overlapping column names should be changed in the joined DataFrame.

In [ ]:
df1.merge(df2, how='inner', on='C', suffixes=('_from_df1', '_from_df2'))

In [ ]:
# go get rid of the above warning cast the overlapping columns to the same types and specify to merge both columns
df1['A'] = df1['A'].astype(float)
df1.merge(df2, how='inner', on=['A', 'C'])

# Working with timeseries data
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html  
The most fundamental building block of timeseries data in pandas is the `Timestamp`. It represents a moment in time to the precision of a nanosecond. It is complemented by `Timedelta`, which represents a stretch of time such as "one month", without being fixed to any date and `Period`, which is a combination of the two such as "June 2018". Yet `Period` needs to have a certain regularity, such as every month.


## Timestamps
Timestamps can be easily created from human readable strings using `pd.datetime`.

In [ ]:
pd.to_datetime('2020-06-09')

In [ ]:
pd.to_datetime('9th June 20')

In [ ]:
pd.to_datetime('06.09.2020')

For non-Americans and people who think that the day should come before the month.

In [ ]:
pd.to_datetime('09.06.2020', dayfirst=True)

In [ ]:
pd.to_datetime('2020-06-09 14:45')

In [ ]:
date = pd.to_datetime('2020-06-09 14:45:30.600700800')
date

`Timestamps` make all information available via attributes.

In [ ]:
date.year

In [ ]:
date.month

In [ ]:
date.day

In [ ]:
date.second

In [ ]:
date.microsecond

In [ ]:
date.nanosecond

Timestamps can be compared:

In [ ]:
date1 = pd.to_datetime('2020-06-09 14:45')
date2 = pd.to_datetime('2020-06-09 14:46')
date1 < date2

When passed a Series, `to_datetime` returns a Series (with the same index), while a list-like is converted to a DatetimeIndex:

In [ ]:
# NaT: Not a Time
pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]))

In [ ]:
pd.to_datetime(['2005/11/23', '2010.12.31'])

`Timestamps` can be formatted using a special set of symbols. All of them can be found here https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
date.strftime('Today is a %A in %B')

In [ ]:
pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M')

### DatetimeIndex

Timestamps can be used to index data.

In [ ]:
index = pd.DatetimeIndex(['2021-05-17', '2021-05-24',
                          '2021-05-31', '2021-06-07',
                          '2020-06-14'])
schedule = pd.Series(['Framing Data with Pandas', 'Basic Visualization',
                      'Cleaning and Analyzing Data with Pandas', 'Advanced Visualization',
                      'Mathematical Programming'], index=index)
schedule

In [ ]:
schedule['2021-05-24':'2021-06-07']

Just like there's NaN for numbers, there's NaT (Not-A-Time) for timestamps:

In [ ]:
dt = pd.to_datetime(['2009/07/31', 'asd'], errors='coerce')
dt

`isnull()` checks for missing dates in DatetimeIndex-objects (NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike):

In [ ]:
dt.isnull()

## Reading timeseries data

In [ ]:
!head data/ao_monthly.txt

Read the data which is in the fixed-width formatted lines format.

In [ ]:
ts = pd.read_fwf('data/ao_monthly.txt', header=None, index_col=0)
ts.head()

This creates an integer index instead of the desired `DateTimeIndex`.

In [ ]:
ts.index

In [ ]:
ts = pd.read_fwf('data/ao_monthly.txt', header=None, index_col=0,
                 parse_dates=[[0, 1]], infer_datetime_format=True)
ts.head()

In [ ]:
ts.index

In [ ]:
ts.plot();

Now that our series is indexed by timestamps, we can aggregate using time related semantics.

In [ ]:
ts.index.year

In [ ]:
ts.groupby(ts.index.year).mean().head()

In [ ]:
ts.groupby(ts.index.year).mean().plot(marker='o');

Using `pd.Grouper` we can specify more complex groupings.

In [ ]:
ts.groupby(pd.Grouper(freq='5Y')).mean().head()
 

An even easier way is to use resample.
The resample function works similar to groupby.
It will group all datetimes into buckets specified by the given [dateoffset](http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) or timedelta and will create a new dataframe after an aggregation functions is called

In [ ]:
ts.resample('5Y').mean().head()

### Resampling
If you do not like the frequency at which your data is sampled you can change the sampling frequency.

In [ ]:
nineteenfifty = ts.loc['1950']
nineteenfifty.head()

In [ ]:
nineteenfifty.plot(marker='o');

In [ ]:
nineteenfifty.asfreq('12D', method='ffill').head()

In [ ]:
nineteenfifty.asfreq('12D', method='ffill').plot(style='--o');

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)

# row 1
nineteenfifty.asfreq('12D').plot(ax=ax[0], style='-o') # no fill
# row 2
nineteenfifty.asfreq('12D', method='ffill').plot(ax=ax[1], marker='o') # forward-fill
nineteenfifty.asfreq('12D', method='bfill').plot(ax=ax[1], style='--o') # back-fill
nineteenfifty.plot(ax=ax[1], style='o') # original

ax[0].legend(['no fill'])
ax[1].legend(['forward-fill', 'back-fill', 'original']);

Downsampling can be done by specifying a smaller frequency.

In [ ]:
nineteenfifty.asfreq('3M', method='ffill').plot(marker='o');

In [ ]:
fig, ax = plt.subplots()

nineteenfifty.asfreq('3M', method='ffill').plot(marker='o', ax=ax) # downsampled
nineteenfifty.plot(ax=ax, style='--o') # original

ax.legend(['3 Month', 'original']);

Lets load some data and see the difference between `resample` and `asfreq`.


In [ ]:
yahoo = pd.read_csv('data/yahoo_stock.csv', index_col=0, parse_dates=True)
yahoo.head()

In [ ]:
ts = yahoo['Close']
ts.plot();

In [ ]:
# 'BA' is code for business year
ts.plot(alpha=0.5, style='-')
ts.resample('BA').mean().plot(style=':')
ts.asfreq('BA').plot(style='--');
plt.legend(['input', 'resample', 'asfreq'], loc='upper left');

### Shifting and Differencing

In [ ]:
ts_resampled = ts.asfreq('D', method='ffill')

Shifting data in time can be done in two ways unsing the `shift` function. Wihtout specifying the `freq` argument shift moves the data. Creating missing values at the one end and losing data at the other. 
In contrast, we can specify the freq, which results in only a shifted index. This only works for datetime indices.

In the below example we shift the data by 365 in the second plot and the index by 365 days in the third plot.

In [ ]:
fig, axes = plt.subplots(nrows=3, sharey=True, figsize=(10, 8))

ts_resampled.plot(ax=axes[0], title='Original')
ts_resampled.shift(365).plot(ax=axes[1], title='shift(365)')
ts_resampled.shift(365, freq='d').plot(ax=axes[2], title='shift(365, freq="d")')

axes[0].axvline('2011', alpha=0.5, color='r', linewidth=3)
axes[1].axvline('2011', alpha=0.5, color='r', linewidth=3)
axes[2].axvline('2011', alpha=0.5, color='r', linewidth=3)

plt.tight_layout()

Shifting is useful for calculations that compare values across timesteps.  

In [ ]:
(ts_resampled - ts_resampled.shift(periods=1)).plot()

For differencing, pandas provides the convenient `diff` method.

In [ ]:
ts_resampled.diff(periods=1).plot()

## Window functions
Window functions are similar to `groupby` as they split data into different groups based on a changing window. The points in each window are aggregated using a summary statistic and then combined back into a timeseries.

### Rolling window
A rolling window is the standard example of a window function. It moves a window of fixed size across the timeseries.

In [ ]:
ts_resampled.plot()
ts_resampled.rolling(365).mean().plot()

Setting `center=True` the point that is aggregated and put into the new series will be from the middle of the window and not from its end. 

In [ ]:
ts_resampled.plot()
ts_resampled.rolling(365, center=True).mean().plot()

### Expanding windows
An expanding window only has a minimal size. Then it grows bigger with each step, taking all previous values into account. This is useful if your timeseries measures a stationary value that only fluctuates around a mean.

In [ ]:
ts_resampled.plot()
ts_resampled.expanding(min_periods=365).mean().plot()

### Exponential weighted windows
An exponential weighted window works like an expanding window, but gives more recent datapoints an exponentially higher weighting in all calculations. Thus it can be viewed as a smooth version of a rolling window.

In [ ]:
ts_resampled.plot()
ts_resampled.ewm(com=50.5, min_periods=5).mean().plot()

There are many more window functions.
(https://pandas.pydata.org/docs/reference/window.html)

## Optional: Timedeltas and Periods
This lecture is already quite packed again. Therefore the last to subchapter are optional. 
You can work through them if you are interested but they are not necessary for the homework.

Timedeltas can be added to Timestamps

In [ ]:
delta = pd.to_timedelta('1 day')
delta

In [ ]:
index = pd.DatetimeIndex(['2021-05-17', '2021-05-24',
                          '2021-05-31', '2021-06-07',
                          '2020-06-14'])
schedule = pd.Series(['Framing Data with Pandas', 'Basic Visualization',
                      'Cleaning and Analyzing Data with Pandas', 'Advanced Visualization',
                      'Mathematical Programming'], index=index)
schedule

In [ ]:
schedule.index += delta
schedule

In [ ]:
pd.to_datetime('2019-08-15') - pd.to_datetime('2018-06-04')

In [ ]:
schedule.index += (pd.to_datetime('2019-08-15') - pd.to_datetime('2018-06-05'))
schedule.index = schedule.index.date
schedule

The combination of Timestamps and Timedeltas allows for nice arithmetics with dates:

In [ ]:
friday = pd.Timestamp('2018-01-05')
saturday = friday + pd.to_timedelta('1 day')
saturday, saturday > friday, saturday - friday

There are even Businessdays in Pandas (Friday --> Monday)

In [ ]:
friday = pd.Timestamp('2018-01-05')
monday = friday + pd.offsets.BDay()
monday

### date_range

In [ ]:
index = pd.DatetimeIndex(['2020-06-16', '2020-06-23',
                          '2020-06-30', '2020-07-07',
                          '2020-07-14'])
schedule = pd.Series(['Statistical Visualization', 'SciPy and Statistical Modeling I',
                      'Statistical Modeling II', 'Creating Experiments',
                      'Performance Optimization'], index=index)
schedule

A more convenient way to create such an index is to use `date_range`.  
`periods` specifies how many entries we want, alternatively we could set an explicit `stop`. `freq` specifies how the entries are spaced. The full list of possible offsets can be found here http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases. Thus the syntax is very similar to `range(start, stop, step)`

In [ ]:
index = pd.date_range('2018-06-04', periods=5, freq='W')
index

Note that `freq='W'` does not mean a simple weekly frequency, but rather `the end of the week for all these dates`.

In [ ]:
index = pd.date_range('2018-06-04', periods=5, freq='7D')
index

Pandas is smart at inferring frequencies:

In [ ]:
tmp = pd.DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], freq='infer')
tmp

In [ ]:
ts = pd.Series(range(len(tmp)), index=tmp)
ts

In [ ]:
ts.resample('D').sum().index

Alternatively we could use a `Period` index to signal that a topic belongs to an entire week.

In [ ]:
prd = pd.Period('2018-06-04', '7D')
prd

In [ ]:
prd.freq

In [ ]:
index = pd.period_range('2018-06-04', periods=5, freq='W')
schedule = pd.Series(['Statistical Visualization', 'SciPy and Statistical Modeling I',
                      'Statistical Modeling II', 'Creating Experiments',
                      'Performance Optimization'], index=index)
schedule

You can easily convert between `Timestamp` and period.

In [ ]:
schedule = schedule.to_timestamp()
schedule

In [ ]:
schedule.to_period(freq='W')

In [ ]:
prd

In [ ]:
prd.to_timestamp().to_period(freq='2D')

### Insertion: Accessing values in Series

In [ ]:
idx = pd.period_range('2000', periods=4)
idx

For Series and Indexes backed by normal NumPy arrays, Series.array will return a new arrays.PandasArray, which is a thin (no-copy) wrapper around a numpy.ndarray. PandasArray isn’t especially useful on its own, but it does provide the same interface as any extension array defined in pandas or by a third-party library.

In [ ]:
idx.array

In [ ]:
pd.Series([1, 2, 3]).array

In [ ]:
idx.to_numpy()

In [ ]:
type(idx.to_numpy()[0])

First info regarding anything to do with timeseries: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

Further:  
A complete time series analysis tutorial. It includes handling time zones plus basic time series prediction and classification.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('zmfe2RaX-14')

# Optional: Exploratory data analysis

Exploratory data analysis (EDA) describes the process of building up an intuition for our data. It is achieved through a combination of data transformations and visualizations. Typical steps in the process of EDA are:


1. Research the fields of the dataset 
2. Form hypotheses/develop investigation themes to explore 
3. Wrangle data 
3. Assess quality of data 
4. Profile data 
5. Explore each individual variable in the dataset 
6. Assess the relationship between each variable and the target 
7. Assess interactions between variables 
8. Explore data across many dimensions 

EDA is very important as we cannot judge whether our modeling makes sense if we don't have intuition for our data. While every analysis starts with EDA you will always return to it when you get new results from modeling.

Here we present pivot tables as an easy way to explore the relationships between variables.

## Pivot for analysis 
Last time we introduced pivot tables as a way to restructure untidy data. However, the original are an operation to create tabular summaries of data. They can be used as a convenient shortcut for a two dimensional groupby.

Let's look at a normal groupby first:

In [ ]:
titanic = pd.read_csv('data/titanic.csv')
embarked_map = pd.DataFrame({'Embarked': ['C', 'Q', 'S'],
                             'EmbarkedLong': ['Cherbourg', 'Queenstown', 'Southampton']})
embarked_map
titanic = titanic.merge(embarked_map).sort_values(by='PassengerId')
titanic

In [ ]:
titanic.groupby('Sex').mean()



Let's say we want to analyze the the influnence of gender and passenger class on survial in the titanic dataset.

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean()

Resetting the index makes it look a little nicer.

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean().reset_index()

For people being used to the tidy format, this can be read intuitively. However, you might still prefer to have the second variable in the column headers. This is what is called a "pivot table".

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean().unstack()

For doing exactly this, pandas provides a shortcut.

In [ ]:
titanic.pivot_table(values='Survived', index='Sex', columns='Pclass')

Pivot tables can also include the margins, i.e. the values aggregated over rows and columns.

In [ ]:
titanic.pivot_table(values='Survived', index='Sex', columns='Pclass', margins=True)

By default `pivot_table` will aggregate using the mean, but we can also choose all the functions available in `groupby` or use our own ones.

In [ ]:
titanic.pivot_table(values='Fare', index='Sex', columns='Pclass', aggfunc=[min, max])

Combining more than two variables is equally possible by stacking them in either the rows or the columns.

In [ ]:
titanic.pivot_table(values='Fare', index=['Sex', 'EmbarkedLong'], columns='Pclass',
                   aggfunc='mean')

In [ ]:
titanic['Age_coarse'] = pd.cut(titanic['Age'], bins=[0, 17, 67, 80], labels=['child', 'grown-up', 'senior'])
titanic['Age_coarse']

The tool [`pivottablejs`](https://github.com/nicolaskruchten/pivottable) allows you qickly explore data with pivotables using drag'n'drop. When using such a graphical tool you should make sure that you turn the interesting things into code so they don't get lost after closing the notebook.

In [ ]:
# pip install pivottablejs
from pivottablejs import pivot_ui
pivot_ui(titanic)

## Profiling
When doing exploratory data analysis, a lot of tasks have to be done every time so they can be automated. Tools like [pandas_profiling](https://github.com/pandas-profiling/pandas-profiling) can create summeries that give insights into many standard questions you can ask to a dataset. However, with abstraction comes less flexibility so tools like this will always only do part of your work and might at times not at all do what you want.

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(titanic)

The following tutorial talks more about tools and processes in exploratory data analysis.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('W5WE9Db2RLU')